In [9]:
import numpy as np
from scipy import linalg
from sklearn.cluster import KMeans

def recursive_spectral_clustering(D, svddim, nocluster, is_dist = True, minsize = 5, maxsize = 100):

    def compute_rbf(Dst, gamma, threshold = 0.01):
        A = np.exp(- gamma * Dst * Dst)
        A = np.where(A < threshold, 0, A)
        return A

    def compute_Lsym(A):
        e = np.ones((A.shape[0], 1))
        d = (A @ e)
        L = A - np.diag(d.flatten())

        d1_2 = np.sqrt(d)
        dr1_2 = np.where(d1_2 == 0, 0, 1/d1_2)

        return ((L / d) / d.T)

    def compute_spectrum(A, svddim):
        Lsym = compute_Lsym(A)
        U, _, _ = linalg.svd(Lsym)

        Uk = U[:, :svddim] 
        s = np.sqrt((Uk * Uk) @ np.ones((svddim, 1)))
        Uk = Uk / s
        return Uk

    def spectral_cluster(Uk, ids, clster, svddim = 100):
        Uki = Uk[ids, :]
        kmeans = KMeans(n_clusters = clster).fit(Uki)
        labels = kmeans.labels_

        cs = [[]] * clster
        for i, l in enumerate(labels):
            cs[l].append(ids[i])
        return cs
    #################
    if is_dist:
        A = compute_rbf(D)
    else:
        A = D
    Lsym = compute_Lsym(A)
    print(Lsym)
    print("Computing spectrum...")
    Uk = compute_spectrum(Lsym, svddim)
    
    clusters = {}
    # Take the whole ids
    ids = [np.arange(Lsym.shape[0])]
    ccount = 0
    while(len(ids) > 0):
        idx = pop(ids)
        clusters = spectral_cluster(Uk, idx, nocluster)
        for c in clusters:
            if len(c) > maxsize:
                ids.append(np.arange(c))
            elif len(c) > minsize:
                clusters[ccount] = c
                ccount += 1
            # else, discard
    return clusters

In [6]:
import pandas as pd
import numpy as np
Adf = pd.read_csv("../data/networks/DREAM_files/dream_3.txt", sep = " ", header = None)
prots = set(Adf[0]).union(Adf[1])
protmap = {k : i for i, k in enumerate(prots)}
Adf[0] = Adf[0].apply(lambda x : protmap[x])
Adf[1] = Adf[1].apply(lambda x : protmap[x])
A = np.zeros((len(prots), len(prots)))
for p, q, w in Adf.values:
    A[int(p+0.2), int(q+0.2)] = w
    A[int(q+0.2), int(p+0.2)] = w
esum = A @ np.ones((A.shape[0], 1))


array([[0.24836601],
       [0.00653595],
       [0.04575163],
       ...,
       [0.04575163],
       [0.00653595],
       [0.06535948]])

In [7]:
for i, s in enumerate(esum):
    if s == 0:
        A[i, i] = 1

In [10]:
out = recursive_spectral_clustering(A, svddim = 50, nocluster = 10, is_dist = False)

[[  -4.02631579    0.            0.         ...    0.
     0.            0.        ]
 [   0.         -153.            0.         ...    0.
     0.            0.        ]
 [   0.            0.          -21.85714286 ...    0.
     0.            0.        ]
 ...
 [   0.            0.            0.         ...  -21.85714286
     0.            0.        ]
 [   0.            0.            0.         ...    0.
  -153.            0.        ]
 [   0.            0.            0.         ...    0.
     0.          -15.3       ]]
Computing spectrum...


/tmp/ipykernel_14360/717584717.py:17: RuntimeWarning: invalid value encountered in sqrt
  d1_2 = np.sqrt(d)
/tmp/ipykernel_14360/717584717.py:18: RuntimeWarning: divide by zero encountered in divide
  dr1_2 = np.where(d1_2 == 0, 0, 1/d1_2)
/tmp/ipykernel_14360/717584717.py:20: RuntimeWarning: divide by zero encountered in divide
  return ((L / d) / d.T)
/tmp/ipykernel_14360/717584717.py:20: RuntimeWarning: invalid value encountered in divide
  return ((L / d) / d.T)


ValueError: array must not contain infs or NaNs